In [17]:
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization, Input
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras import regularizers, optimizers, applications
import pandas as pd
import numpy as np

In [2]:
insta_meta = pd.read_csv('recipes.csv', sep=';')
columns_tag = [i for i in insta_meta.columns if 'tag' in i]
insta_meta_tags = insta_meta[columns_tag + ["photo_id"]]

In [3]:
insta_meta_tags.loc[:,'photo_id'] = insta_meta_tags.loc[:,"photo_id"] + '.png'

C:\Users\u0143591\Anaconda3\envs\keras\lib\site-packages\pandas\core\indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [4]:
insta_meta_tags

,tag_healthy,tag_tasty,tag_dinner,tag_vegan,tag_breakfast,tag_lunch,tag_dessert,tag_baking,tag_cake,tag_easyrecipes,...,tag_oatmeal,tag_natural,tag_beef,tag_noodles,tag_italian,tag_drinks,tag_steak,tag_sandwich,tag_strawberry,photo_id
0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CMPELQqh_nh.png
1,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CMPEHqppR-t.png
2,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,CMPEBY9Buko.png
3,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CMPEAvhF9i1.png
4,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,CMPD73jJssJ.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11691,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,CMsOtSZhtSx.png
11692,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,CMsOstJAXEJ.png
11693,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CMsOrPOrSkl.png
11694,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,CMsOnHuFxoF.png


In [ ]:
ImageDataGenerator().flow_from_dataframe()

In [52]:
df = insta_meta_tags.copy()

datagen=ImageDataGenerator(rescale=1./255.,
                           rotation_range=10,
                           width_shift_range=0.2,
                           height_shift_range=0.2,
                           shear_range=0.1,
                           zoom_range=0.2,
                           horizontal_flip=True)

test_datagen=ImageDataGenerator(rescale=1./255.)

train_generator=datagen.flow_from_dataframe(dataframe=df[:1000], #int(len(insta_meta_tags) * 0.8)],
                                            directory="./recipes",
                                            x_col="photo_id",
                                            y_col=columns_tag,
                                            batch_size=32,
                                            seed=42,
                                            shuffle=True,
                                            class_mode="raw",
                                            save_to_dir="./recipes_augmented") # we save augmented images to add in report

valid_generator=test_datagen.flow_from_dataframe(dataframe=df[1000:1200], #df[int(len(insta_meta_tags) * 0.8):int(len(insta_meta_tags) * 0.9)],
                                                 directory="./recipes",
                                                 x_col="photo_id",
                                                 y_col=columns_tag,
                                                 batch_size=32,
                                                 seed=42,
                                                 shuffle=True,
                                                 class_mode="raw")

test_generator=test_datagen.flow_from_dataframe(dataframe=df[1200:1400],#df[int(len(insta_meta_tags) * 0.9):],
                                                directory="./recipes",
                                                x_col="photo_id",
                                                batch_size=1,
                                                seed=42,
                                                shuffle=False,
                                                class_mode=None)

Found 1000 validated image filenames.
Found 200 validated image filenames.
Found 200 validated image filenames.


In [53]:
# custom
#inp = Input(shape = (256,256,3))
#x = Conv2D(32, (3, 3), padding = 'same')(inp)
#x = Activation('relu')(x)
#x = Conv2D(32, (3, 3))(x)
#x = Activation('relu')(x)
#x = MaxPooling2D(pool_size = (2, 2))(x)
#x = Dropout(0.25)(x)
#x = Conv2D(64, (3, 3), padding = 'same')(x)
#x = Activation('relu')(x)
#x = Conv2D(64, (3, 3))(x)
#x = Activation('relu')(x)
#x = MaxPooling2D(pool_size = (2, 2))(x)
#x = Dropout(0.25)(x)
#x = Flatten()(x)
#x = Dense(512)(x)
#x = Activation('relu')(x)
#x = Dropout(0.5)(x)
#outputs = [Dense(1, activation = 'softmax')(x) for i in range(len(columns_tag))]
#model = Model(inp, outputs)
#model.compile(optimizers.Adam(lr = 0.0001, decay = 1e-6),
#              loss = ["binary_crossentropy" for i in range(len(columns_tag))],
#              metrics = ["accuracy"])

In [54]:
# ResNet-50
resn_base = applications.ResNet50(include_top=False, weights='imagenet', input_shape=(256,256,3))
resn_base.trainable = False # freeze the trainable layers of the ResNet-50
# we provide our inputs to the base_model and explicitly set training to false such that this part of the network is not trained
x = resn_base(inp, training=False)
x = Conv2D(1024, (2,2), activation='relu', padding='same')(x)
#x = Conv2D(512, (2,2), activation='relu', padding='same')(x)
x = GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512)(x)
x = Activation('relu')(x)
outputs = [Dense(1, activation = 'sigmoid')(x) for i in range(len(columns_tag))]
model = Model(inp, outputs)
model.compile(optimizers.Adam(lr = 0.001, decay = 1e-06),
              loss = ["binary_crossentropy" for i in range(len(columns_tag))],
              metrics = ["accuracy"])

In [55]:
model.summary()

Model: "functional_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 8, 8, 2048)   23587712    input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_9 (Conv2D)               (None, 8, 8, 1024)   8389632     resnet50[0][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_10 (Gl (None, 1024)         0           conv2d_9[0][0]                   
______________________________________________________________________________________

In [56]:
def generator_wrapper(generator):
    for batch_x,batch_y in generator:
        yield (batch_x,[batch_y[:,i] for i in range(len(columns_tag))])

In [43]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
result = model.fit(x=generator_wrapper(train_generator),
                   steps_per_epoch=STEP_SIZE_TRAIN,
                   validation_data=generator_wrapper(valid_generator),
                   validation_steps=STEP_SIZE_VALID,
                   epochs=10,verbose=1
                  )

3/3 [==============================] - 15s 5s/step - loss: 1441.8472 - dense_787_loss: 13.8196 - dense_788_loss: 12.7077 - dense_789_loss: 12.2312 - dense_790_loss: 13.9785 - dense_791_loss: 13.8196 - dense_792_loss: 12.7077 - dense_793_loss: 14.2962 - dense_794_loss: 14.9315 - dense_795_loss: 14.7727 - dense_796_loss: 14.4550 - dense_797_loss: 14.7727 - dense_798_loss: 14.4550 - dense_799_loss: 13.9785 - dense_800_loss: 13.8196 - dense_801_loss: 14.4550 - dense_802_loss: 14.4550 - dense_803_loss: 15.2492 - dense_804_loss: 14.9315 - dense_805_loss: 14.9315 - dense_806_loss: 14.6139 - dense_807_loss: 14.9315 - dense_808_loss: 14.2962 - dense_809_loss: 15.0904 - dense_810_loss: 14.4550 - dense_811_loss: 15.0904 - dense_812_loss: 15.2492 - dense_813_loss: 14.1373 - dense_814_loss: 15.0904 - dense_815_loss: 14.7727 - dense_816_loss: 14.9315 - dense_817_loss: 15.2492 - dense_818_loss: 14.4550 - dense_819_loss: 15.0904 - dense_820_loss: 14.9315 - dense_821_loss: 14.9315 - dense_822_loss: 15.

In [47]:
test_generator.reset()
pred=model.predict(test_generator,
                   steps=STEP_SIZE_TEST,
                   verbose=1)

50/50 [==============================] - 2s 44ms/step


In [49]:
pred

[array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]], dtype=float32),
 array([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
   

In [13]:
[Dense(1, activation = 'sigmoid') for i in range(len(columns_tag))]

97